<a href="https://colab.research.google.com/github/SiheeNoh/DFC606/blob/master/HW4_2019517012_%EB%85%B8%EC%8B%9C%ED%9D%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW4 - Data Analysis & Make Prediction Models

#### Data Science in Korea University, Prof. Jaewoo Kang
#### COSE471, Spring 2020
#### Due : 7/01 (wednes)  11:59 PM [Through Black board]

## Assignment Credit

### Your assignments are 40% of the total score. The scores for each assignment are as follows.
- __HW#1: 5%__
- __HW#2: 10%__
- __HW#3: 10%__
- __HW#4: 15%__

## Python Assingment HW4 Guide
- You need to fill out your code in ###YOUR CODE HERE### or #TODO.
- The code results should be the same as the results displayed below the code or the format we gave in problem description.
- When solving descriptive problems, both __Korean and English are available.__
- The some libraries are already imported below. __But you should import more.__
    - If you don't have some libraries, install it with pip or conda command. [Google will help you]

## Problem1. Scikit-learn
- Scikit-learn is the most common python library in machine learning.
- Let's port our work from matplotlib to scikit-learn.
- However, not all plots will be handled. We will focus on the most recent data, 2010-2015.

<b>URL:</b> http://scikit-learn.org/
<br/><b>Install guide:</b> http://scikit-learn.org/stable/install.html

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import time
from datetime import datetime
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import nltk
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Problem1. Regression Model
## -Amzon stock price prediction.

### Problem 1-1
- In this problem you will predict stock price only with price related data. People usually call it as Technical analysis.
- Find best combination of parameters to predict next day price.
- __Your MSE loss should be lower than 30.__

In [5]:
%%time
!rm -f AMZN.csv covid_all.csv covid_twitter.csv OxCGRT_latest.csv train.csv

!wget -nc https://raw.githubusercontent.com/SiheeNoh/DFC606/master/AMZN.csv
!wget -nc https://raw.githubusercontent.com/SiheeNoh/DFC606/master/covid_all.csv
!wget -nc https://raw.githubusercontent.com/SiheeNoh/DFC606/master/covid_twitter.csv
!wget -nc https://raw.githubusercontent.com/SiheeNoh/DFC606/master/OxCGRT_latest.csv
!wget -nc https://raw.githubusercontent.com/SiheeNoh/DFC606/master/train.csv

--2020-07-01 14:54:21--  https://raw.githubusercontent.com/SiheeNoh/DFC606/master/AMZN.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375864 (367K) [text/plain]
Saving to: ‘AMZN.csv’

AMZN.csv            100%[===================>] 367.05K  --.-KB/s    in 0.06s   

2020-07-01 14:54:21 (6.37 MB/s) - ‘AMZN.csv’ saved [375864/375864]

--2020-07-01 14:54:25--  https://raw.githubusercontent.com/SiheeNoh/DFC606/master/covid_all.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7518135 (7.2M) [text/plain]
Saving to: ‘covid_all.csv’

covid_al

KeyboardInterrupt: ignored

In [ ]:
# TODO : load data 'AMZN.csv'
#Hint: use pd.read_csv and set read_csv function paramete 'index_col' to set Date  variable to data` Index.
amzn = pd.read_csv("AMZN.csv")

In [ ]:
amzn.head()

<br/><br/>
- We will use only __one kind of variable__ for __last 'lookback' days__ to predict next day close price.<br/>
For example, if we set lookback days as 5, and use close price for our independent variable.<br/><br/>
Close price of [1997-05-15, 1997-05-16, 1997-05-19, 1997-05-20, 1997-05-21] --> predict [1997-05-22] <br/><br/>

In [ ]:
"""
TODO : make get_concated_data function.
       this function create x and y data

@Function
get_concated_data

@Inputs
__
raw_data : dataframe, full Amazon stock data.
     

@Returns
___

x_data 
    : <class 'numpy.ndarray'>
    : crteated data should in shape of [len(raw_data)-lookback, lookback]
    : x_data should be made based on 'col_name', parameters of the function, variable we will use
    
y_data 
    : <class 'numpy.ndarray'>
    : crteated data should in shape of [len(raw_data)-lookback]
    : y should be always made based on 'Close'.
"""


In [ ]:
amzn.head()

In [ ]:
# We will use only one kind of variable as a time series data
def get_concated_data(raw_data, col_name, lookback):
    x_datas = list()
    y_datas = list()
    # col_name : Variable to be used
    col_data = raw_data[col_name].values
    # aggregate target variable data for length of lookback days
    
    ##### Your Code Here ####
    ##HInt: use for function and append function in for function x,y data to x_datas,y_datas
    
    # crteated data should in shape of [len(raw_data)-lookback, lookback]
    # x_data should be made based on 'col_name', parameters of the function, variable we will use
    x_datas = np.zeros([len(raw_data)-lookback, lookback])

    for i in range(0, lookback):
        x_datas[:, i] = raw_data[col_name].shift(lookback-i)[lookback:len(raw_data)].values
        #x_datas.append(raw_data[col_name][i-lookback:i])
    y_datas = raw_data['Close'][-(len(raw_data)-lookback):]
    #########################    
    x_data = np.concatenate(x_datas,0)
    y_data = np.asarray(y_datas)

    return x_data, y_data

In [ ]:
"""
TODO : make split_data function which split train / test (last 60 data, approximately 3 month)

@Function
split_datas

@Inputs
__
data : numpy array
     
@Returns
___
x_train, x_test, y_train, y_test
    :<class 'numpy.ndarray'>

"""

In [ ]:
def split_datas(x,y):
    ##### Your Code Here ####
    ##hint: for time series data yo should not split data randomly ,just split data '0:your split parameter' , 'split parameter: last'
    ratio = int(len(x) / len(y))
    x_train = x[:-60*ratio]
    y_train = y[:-60]
    x_test = x[-60*ratio:]
    y_test = y[-60:]
    ###########################
    return x_train, x_test, y_train, y_test

In [ ]:
x, y = get_concated_data(amzn, 'Close', 5)
x[:10], y[:10]

In [ ]:
x_train, x_test, y_train, y_test = split_datas(x, y)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

### <br/>Problem 1-2 Find best parameter combination 
- You can choose which __columns to use__ as variable for __how many days__, and in which __polynomial space.__
- Find the best combination of parameters to predict next day price.
- __Find combination which makes MSE loss for train data lower than 6.__

In [ ]:
# TODO : complete run_regression function
# Hint : use api i gave below

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

def run_regression(raw_data, column_name, lookback_days, poly_degree):
    #print(column_name, lookback_days, poly_degree)
    # performing regression analysis with specified parameters
    x, y = get_concated_data(raw_data, column_name, lookback_days)
    x_train, x_test, y_train, y_test = split_datas(x, y)
    
    ##### Your Code Here ####
    PolynomialRegression = PolynomialFeatures(degree = poly_degree)
    poly = PolynomialRegression.fit_transform(np.reshape(x, (len(y), lookback_days)))

    X_train = PolynomialRegression.fit_transform(np.reshape(x_train, (len(y_train), lookback_days)))
    X_test = PolynomialRegression.fit_transform(np.reshape(x_test, (len(y_test), lookback_days)))
    
    model = LinearRegression().fit(X_train, y_train) #fit the model
    train_pred = model.predict(X_train) # for train which data you have to use?
    test_pred = model.predict(X_test) # for test which data you have to use?
    
    print ("MSE - Train %.4f"%mean_squared_error(model.predict(X_train), y_train))
    print ("MSE - Test %.4f"%mean_squared_error(model.predict(X_test), y_test))
    
    return test_pred, y_test, mean_squared_error(model.predict(X_train), y_train), mean_squared_error(model.predict(X_test), y_test)

In [ ]:
amzn.columns

In [ ]:
# TODO : run experiment with various combination
best_pred = 0
best_test = 0
best_column = ''
best_lookback_day = 0
best_poly_degree = 0
best_error = 0
best_list = []

for lookback_day in range(10, 15):
    if len(best_list) > 0:
        break

    for column in amzn.columns[1:]:
        if len(best_list) > 0:
            break
            
        for poly_degree in range(1, 5):            
            test_pred, y_test, train_error, test_error = run_regression(amzn, column_name=column, lookback_days=lookback_day, poly_degree=poly_degree)
            if train_error < 6:
                best_column = column
                best_lookback_day = lookback_day
                best_poly_degree = poly_degree
                best_pred = test_pred
                best_error = test_error
                best_list.append([ column, lookback_day, poly_degree, train_error, test_error])
                break

In [ ]:
 best_column, best_lookback_day, best_poly_degree

Write Combination that you've found. (Train loss lower than 6) <br/><br/>
Variable :    Close   <br/>
Lookback :    11    <br/>
Poly_degree :  4     <br/>

### Problem 1-3 Overfitting & Underfitting
1. Briefly explain about the concept of overfitting and underftting with proper example that you can find from experiment above.
2. In the above problem desciption, I set your goal as fiding best parameters for total data loss. What is wrong with it? 

### YOUR ANSWER HERE ###

- 1. Train Error가 0.80, Test Error 52713042603.32 로 학습된 모델의 오차는 적어도 테스트 데이터의 에러 크기 워낙 커서 오버피팅 되었다.
- 2. Train Error가 아닌 Test Error가 적게 나오는 파라미터를 찾는 로직으로 수정해야 한다.
      

### Problem 1-4 Visualization
- Visualize your predicton.
- You can use code below with proper variable.
- __Your best model's prediction --> best_pred.__

In [ ]:
# TODO : run experiment with various combination
best_pred = 0
best_test = 0
best_column = ''
best_lookback_day = 0
best_poly_degree = 0
best_error = -1
best_list = []

for lookback_day in range(1, 15):
    #for column in raw_data.columns[1:]:
    for column in ['Close', 'Adj Close']:
        for poly_degree in range(1, 5):            
            test_pred, y_test, error, test_error = run_regression(amzn, column_name=column, lookback_days=lookback_day, poly_degree=poly_degree)
            if best_error < 0 or test_error < best_error:
                best_column = column
                best_lookback_day = lookback_day
                best_poly_degree = poly_degree
                best_pred = test_pred
                best_error = test_error
                best_list.append([ column, lookback_day, poly_degree, error, test_error])
                #break

In [ ]:
# TODO : Assign your best prediction for test data to best_pred
print('Variable:', best_column)
print('Lookback:', best_lookback_day)
print('Poly_degree:', best_poly_degree)

best_pred, y_besty, b_error, best_test_error = run_regression(raw_data, best_column, best_lookback_day, best_poly_degree)


In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(20,10))
plt.plot(best_pred, label='prediction')
plt.plot(y_test, label='target')
plt.legend(loc='best', fontsize=20)
plt.show()

## Problem2. Regression Model
## -Covid Confirmed prediction.

In [ ]:
# #import covidall dataset 
# train = pd.read_csv('../train.csv')
# all_data = pd.read_csv('../covid_all.csv') #set your working directory
# all_data

In [ ]:
#import covidall dataset 
train = pd.read_csv("train.csv", sep=",")
all_data = pd.read_csv("covid_all.csv", sep=",")
all_data.head()

In [ ]:
country_dict = {'Afghanistan': 0,
 'Albania': 1,
 'Algeria': 2,
 'Andorra': 3,
 'Angola': 4,
 'Antigua and Barbuda': 5,
 'Argentina': 6,
 'Armenia': 7,
 'Australia': 8,
 'Austria': 9,
 'Azerbaijan': 10,
 'Bahamas': 11,
 'Bahrain': 12,
 'Bangladesh': 13,
 'Barbados': 14,
 'Belarus': 15,
 'Belgium': 16,
 'Belize': 17,
 'Benin': 18,
 'Bhutan': 19,
 'Bolivia': 20,
 'Bosnia and Herzegovina': 21,
 'Botswana': 22,
 'Brazil': 23,
 'Brunei': 24,
 'Bulgaria': 25,
 'Burkina Faso': 26,
 'Burma': 27,
 'Burundi': 28,
 'Cabo Verde': 29,
 'Cambodia': 30,
 'Cameroon': 31,
 'Canada': 32,
 'Central African Republic': 33,
 'Chad': 34,
 'Chile': 35,
 'China': 36,
 'Colombia': 37,
 'Congo (Brazzaville)': 38,
 'Congo (Kinshasa)': 39,
 'Costa Rica': 40,
 "Cote d'Ivoire": 41,
 'Croatia': 42,
 'Cuba': 43,
 'Cyprus': 44,
 'Czechia': 45,
 'Denmark': 46,
 'Diamond Princess': 47,
 'Djibouti': 48,
 'Dominica': 49,
 'Dominican Republic': 50,
 'Ecuador': 51,
 'Egypt': 52,
 'El Salvador': 53,
 'Equatorial Guinea': 54,
 'Eritrea': 55,
 'Estonia': 56,
 'Eswatini': 57,
 'Ethiopia': 58,
 'Fiji': 59,
 'Finland': 60,
 'France': 61,
 'Gabon': 62,
 'Gambia': 63,
 'Georgia': 64,
 'Germany': 65,
 'Ghana': 66,
 'Greece': 67,
 'Grenada': 68,
 'Guatemala': 69,
 'Guinea': 70,
 'Guinea-Bissau': 71,
 'Guyana': 72,
 'Haiti': 73,
 'Holy See': 74,
 'Honduras': 75,
 'Hungary': 76,
 'Iceland': 77,
 'India': 78,
 'Indonesia': 79,
 'Iran': 80,
 'Iraq': 81,
 'Ireland': 82,
 'Israel': 83,
 'Italy': 84,
 'Jamaica': 85,
 'Japan': 86,
 'Jordan': 87,
 'Kazakhstan': 88,
 'Kenya': 89,
 'Korea, South': 90,
 'Kosovo': 91,
 'Kuwait': 92,
 'Kyrgyzstan': 93,
 'Laos': 94,
 'Latvia': 95,
 'Lebanon': 96,
 'Liberia': 97,
 'Libya': 98,
 'Liechtenstein': 99,
 'Lithuania': 100,
 'Luxembourg': 101,
 'MS Zaandam': 102,
 'Madagascar': 103,
 'Malawi': 104,
 'Malaysia': 105,
 'Maldives': 106,
 'Mali': 107,
 'Malta': 108,
 'Mauritania': 109,
 'Mauritius': 110,
 'Mexico': 111,
 'Moldova': 112,
 'Monaco': 113,
 'Mongolia': 114,
 'Montenegro': 115,
 'Morocco': 116,
 'Mozambique': 117,
 'Namibia': 118,
 'Nepal': 119,
 'Netherlands': 120,
 'New Zealand': 121,
 'Nicaragua': 122,
 'Niger': 123,
 'Nigeria': 124,
 'North Macedonia': 125,
 'Norway': 126,
 'Oman': 127,
 'Pakistan': 128,
 'Panama': 129,
 'Papua New Guinea': 130,
 'Paraguay': 131,
 'Peru': 132,
 'Philippines': 133,
 'Poland': 134,
 'Portugal': 135,
 'Qatar': 136,
 'Romania': 137,
 'Russia': 138,
 'Rwanda': 139,
 'Saint Kitts and Nevis': 140,
 'Saint Lucia': 141,
 'Saint Vincent and the Grenadines': 142,
 'San Marino': 143,
 'Sao Tome and Principe': 144,
 'Saudi Arabia': 145,
 'Senegal': 146,
 'Serbia': 147,
 'Seychelles': 148,
 'Sierra Leone': 149,
 'Singapore': 150,
 'Slovakia': 151,
 'Slovenia': 152,
 'Somalia': 153,
 'South Africa': 154,
 'South Sudan': 155,
 'Spain': 156,
 'Sri Lanka': 157,
 'Sudan': 158,
 'Suriname': 159,
 'Sweden': 160,
 'Switzerland': 161,
 'Syria': 162,
 'Taiwan*': 163,
 'Tanzania': 164,
 'Thailand': 165,
 'Timor-Leste': 166,
 'Togo': 167,
 'Trinidad and Tobago': 168,
 'Tunisia': 169,
 'Turkey': 170,
 'US': 171,
 'Uganda': 172,
 'Ukraine': 173,
 'United Arab Emirates': 174,
 'United Kingdom': 175,
 'Uruguay': 176,
 'Uzbekistan': 177,
 'Venezuela': 178,
 'Vietnam': 179,
 'West Bank and Gaza': 180,
 'Western Sahara': 181,
 'Zambia': 182,
 'Zimbabwe': 183}

### <br/>Problem 2-1 Plot ' Spain' covid-Confiremedcases graph between march 1 to march 11.
- your completed graph doesn't have to be the same as the example.

![image.png](attachment:image.png)

In [ ]:
# plot ' Spain' covid-Confiremedcases graph between march 1 to march 11.
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,6))
y1 = all_data[(all_data['Country_Region']==country_dict['Spain']) & (all_data['Day_num']>39) & (all_data['Day_num']<=49)][['ConfirmedCases']]
x1 = range(0, len(y1))
ax1.plot(x1, y1, 'bo--')
ax1.set_title("Spain ConfirmedCases between days 39 and 49")
ax1.set_xlabel("Days")
ax1.set_ylabel("ConfirmedCases")

## To do ###
# Hint : using the above process but you have to transform your variable to log scale.
##apply log transform to y variables ## 
x2 = x1
y2 = np.log(y1)
ax2.plot(x2, y2, 'bo--')
ax2.set_title("Spain ConfirmedCases between days 39 and 49")
ax2.set_xlabel("Days")
ax2.set_ylabel("ConfirmedCases")


In [ ]:
##don`t touch this cell##

# Filter selected features
data = all_data.copy()
features = ['Id', 'ForecastId', 'Country_Region', 'Province_State', 'ConfirmedCases', 'Fatalities', 
       'Day_num']
data = data[features]

# Apply log transformation to all ConfirmedCases and Fatalities columns, except for trends
data[['ConfirmedCases', 'Fatalities']] = data[['ConfirmedCases', 'Fatalities']].astype('float64')
data[['ConfirmedCases', 'Fatalities']] = data[['ConfirmedCases', 'Fatalities']].apply(lambda x: np.log1p(x))

# Replace infinites
data.replace([np.inf, -np.inf], 0, inplace=True)


### <br/>Problem 2-2  complete below function to  Split data into train/test and apply linear regression

In [ ]:
#  Split data into train/test
def split_data(df, train_lim, test_lim):
    df.loc[df['Day_num']<=train_lim , 'ForecastId'] = -1
    df = df[df['Day_num']<=test_lim]
    
    ##To do ## for Train set 
    x_train = df[(df['Day_num']<=train_lim)].drop(['ConfirmedCases', 'Fatalities'], axis=1)
    y_train_1 = df.loc[(df['Day_num']<=train_lim), 'ConfirmedCases']
    y_train_2 = df.loc[(df['Day_num']<=train_lim), 'Fatalities']
    
    ##To do ## for  Test set
    x_test = df[(df['Day_num']>train_lim)].drop(['ConfirmedCases', 'Fatalities'], axis=1)
    
    # Clean Id columns and keep ForecastId as index
    x_train.drop('Id', inplace=True, errors='ignore', axis=1)
    x_train.drop('ForecastId', inplace=True, errors='ignore', axis=1)

    x_test.drop('Id', inplace=True, errors='ignore', axis=1)
    x_test.drop('ForecastId', inplace=True, errors='ignore', axis=1)

    return x_train, y_train_1, y_train_2, x_test #check to devide y_dataset 

# Linear regression model
def lin_reg(X_train, Y_train, X_test):
    ##To do ##   Create linear regression object
    regr = linear_model.LinearRegression()
    
    ## To do ##  Train the model using the training sets
    regr.fit(X_train, Y_train)
    
    # To do ## Make predictions using the testing set
    y_pred = regr.predict(X_test)
    
    return regr, y_pred

In [ ]:
##don`t touch this cell##
dates_list = ['2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09', 
                 '2020-03-10', '2020-03-11','2020-03-12','2020-03-13','2020-03-14','2020-03-15','2020-03-16','2020-03-17','2020-03-18',
                 '2020-03-19','2020-03-20','2020-03-21','2020-03-22','2020-03-23', '2020-03-24', '2020-03-25', '2020-03-26', '2020-03-27', 
                 '2020-03-28', '2020-03-29', '2020-03-30', '2020-03-31', '2020-04-01', '2020-04-02', '2020-04-03', '2020-04-04', '2020-04-05', 
                 '2020-04-06', '2020-04-07', '2020-04-08', '2020-04-09', '2020-04-10', '2020-04-11', '2020-04-12', '2020-04-13', '2020-04-14','2020-04-15','2020-04-16','2020-04-17','2020-04-18','2020-04-19',
                '2020-04-20','2020-04-21','2020-04-22','2020-04-23','2020-04-24','2020-04-25','2020-04-26','2020-04-27','2020-04-28','2020-04-29','2020-04-30',
                '2020-05-01','2020-05-02','2020-05-03','2020-05-04','2020-05-05','2020-05-06','2020-05-07','2020-05-08','2020-05-09','2020-05-10','2020-05-11',
                '2020-05-12','2020-05-13', '2020-05-14']

def plot_linreg_basic_country(data, country_name, dates_list, day_start, shift, train_lim, test_lim):
    
    data_country = data[data['Country_Region']==country_dict[country_name]]
    data_country = data_country.loc[data_country['Day_num']>=day_start]
    X_train, Y_train_1, Y_train_2, X_test = split_data(data_country, train_lim, test_lim)
    model, pred = lin_reg(X_train, Y_train_1, X_test)
    
    # Create a df with both real cases and predictions (predictions starting on March 12th)
    X_train_check = X_train.copy()
    X_train_check['Target'] = Y_train_1
    
    X_test_check = X_test.copy()
    X_test_check['Target'] = pred
    
    X_final_check = pd.concat([X_train_check, X_test_check])
    
    # Select predictions from March 1st to March 25th
    predicted_data = X_final_check.loc[(X_final_check['Day_num'].isin(list(range(day_start, day_start+len(dates_list)))))].Target
    real_data = train.loc[(train['Country_Region']==country_name) & (train['Date'].isin(dates_list))]['ConfirmedCases']
    dates_list_num = list(range(0,len(dates_list)))
    
    # Plot results
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,6))
    
    ax1.plot(dates_list_num, np.expm1(predicted_data))
    ax1.plot(dates_list_num, real_data)
    ax1.axvline(30-shift, linewidth=2, ls = ':', color='grey', alpha=0.5)
    
    ax1.legend(['Predicted cases', 'Actual cases', 'Train-test split'], loc='upper left')
    ax1.set_xlabel("Day count (from March " + str(1+shift) + " to March 25th)")
    ax1.set_ylabel("Confirmed Cases")
    
    ax2.plot(dates_list_num, predicted_data)
    ax2.plot(dates_list_num, np.log1p(real_data))
    ax2.axvline(30-shift, linewidth=2, ls = ':', color='grey', alpha=0.5)
    ax2.legend(['Predicted cases', 'Actual cases', 'Train-test split'], loc='upper left')
    ax2.set_xlabel("Day count (from March " + str(1+shift) + " to March 30th)")
    ax2.set_ylabel("Log Confirmed Cases")

    plt.suptitle(("ConfirmedCases predictions based on Log-Lineal Regression for "+country_name))

### <br/>Problem 2-3  using above function'plot_linreg_basic_country' run the Linear Regression workflow in 'Korea, South' case
 - your completed graph doesn't have to be the same as the example.


In [ ]:
## To do ##  
country_name = 'Korea, South'
march_day = 1
day_start = 39
dates_list2 = dates_list[march_day:]
train_lim, test_lim = 69, 112
plot_linreg_basic_country(data, country_name, dates_list2, day_start, march_day, train_lim, test_lim)

굵은 텍스트![image.png](attachment:image.png)

### <br/>Problem 2-4 Describe the significance and limitations of the graph above.

- Day Count가 30미만에서 데이터를 잘 학습하다가 그 이후 에러가 크게 증가한다.
  즉, 오버피팅 문제가 발생함
  
- 오버피팅 해결방안
   1. 더 많은 데이터셋으로 모델링
   2. 피처수를 작게하여 테스트 에러를 줄이는 모델을 찾는다.
   3. 학습 기간을 늘린다.

![대체 텍스트](https://)학.

## 3. Caculating Stringency Indexes of six countries 

Reference : https://www.bsg.ox.ac.uk/sites/default/files/Calculation%20and%20presentation%20of%20the%20Stringency%20Index.pdf

Look at the document above and learn about Stringency Index (not legacy stringency index!!)

## 3-1 Data filtering

In [ ]:
#load csv file you care ("")
###YOUR CODE HERE###
latest = pd.read_csv("OxCGRT_latest.csv", sep=",")
latest.to_csv("data.csv", sep=",")

In [ ]:
#Filter rows with six unique countries by 'CountryName'
#('CountryName' == Australia, China, Japan, New Zealand, Singapore, South Korea)
###YOUR CODE HERE###
OxCGRT =  latest[latest['CountryName'].isin(['Australia', 'China', 'Japan', 'New Zealand', 'Singapore', 'South Korea'])]
OxCGRT.head()

In [ ]:
#Drop columns you don't need
#print dataframe you care about
###YOUR CODE HERE###
#use columns : 

columns = ['CountryName', 'Date', 'C1_School closing', 'C1_Flag', 'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events','C3_Flag',
        'C4_Restrictions on gatherings','C4_Flag','C5_Close public transport','C5_Flag','C6_Stay at home requirements','C6_Flag',
        'C7_Restrictions on internal movement','C7_Flag','C8_International travel controls','H1_Public information campaigns','H1_Flag']

OxCGRT = OxCGRT[columns]
OxCGRT = OxCGRT.reset_index(drop=True)
OxCGRT.head()

## 3-2 Calculating Stringency Index

you can use some pakages to deal with 'nan' values

In [ ]:
OxCGRT.fillna(0, inplace=True)

In [ ]:
#stringency calculation
#step1 : calculate weight for a policy of general scope
###YOUR CODE HERE###
OxCGRT["W"] = 0

for i in range(0, len(OxCGRT)):
    result = 0
    for column in ['C1_School closing', 'C2_Workplace closing', 'C3_Cancel public events',
                    'C4_Restrictions on gatherings', 'C5_Close public transport',
                    'C6_Stay at home requirements',  'C7_Restrictions on internal movement',
                    'H1_Public information campaigns']:
        result += 1/(OxCGRT.loc[i, column]+1)/8
        #print(result)
    OxCGRT.loc[i, "W"] = result
    #print(data.loc[i, "W"])

In [ ]:
OxCGRT.head()

In [ ]:
#step2 : calculate sub-indices
#calculate C1-C9, G1-G8, and I1-I9 
###YOUR CODE HERE###
OxCGRT['C1'] = OxCGRT['C1_School closing']
OxCGRT['C2'] = OxCGRT['C2_Workplace closing']
OxCGRT['C3'] = OxCGRT['C3_Cancel public events']
OxCGRT['C4'] = OxCGRT['C4_Restrictions on gatherings']
OxCGRT['C5'] = OxCGRT['C5_Close public transport']
OxCGRT['C6'] = OxCGRT['C6_Stay at home requirements']
OxCGRT['C7'] = OxCGRT['C7_Restrictions on internal movement']
OxCGRT['C8'] = OxCGRT['C8_International travel controls']
OxCGRT['C9'] = OxCGRT['H1_Public information campaigns']
OxCGRT['C9_Flag'] = OxCGRT['H1_Flag']

CFlgas = ["C1_Flag", "C2_Flag", "C3_Flag", "C4_Flag", "C5_Flag", "C6_Flag", "C7_Flag", "H1_Flag"]

#numbers = range(1, 9)
Ns = [3,3,2,4,2,4,2,2]
for i in range(1, 9):
    OxCGRT['G'+str(i+1)] = 1
    OxCGRT['I'+str(i+1)] = 0.0       
    
for i in range(0, len(OxCGRT)):
    result = 0
        
    for j in range(0, 8):        
        w = OxCGRT.loc[i, "W"]
        C = OxCGRT.loc[i, CFlgas[j]]
        N = Ns[j]
        G = OxCGRT.loc[i, "G"+str(j+1)]
        I = 100 * (  C * (1-w) / N + w * G)
        
        OxCGRT.loc[i, "I"+str(j+1)] = I

OxCGRT['I9'] = 100 * OxCGRT['C8_International travel controls'] / 4 

In [ ]:
#step3 : calculate stringency index I (up to 2 decimal points)
###YOUR CODE HERE###
stringency_index = (OxCGRT[['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7','I8', 'I9']]).mean(axis=1)
OxCGRT['NULL'] = OxCGRT[['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7','I8', 'I9']].isnull().sum(axis=1)
stringency_index = round(stringency_index, 2)
stringency_index

In [ ]:
#step4 : make your stingency index array into dataframe(column name = 'my_StringencyIndex') and concat it(axis = 1) to the result of 1-1 
###YOUR CODE HERE###
OxCGRT['StringencyIndex'] = OxCGRT6['StringencyIndex']
OxCGRT.loc[OxCGRT['NULL'] < 6, "my_StringencyIndex"] = stringency_index


## Problem4. Topic Modeling (Clustering analysis of documents) for tweets related to COVID-19
A **topic model** is a type of statistical model for discovering the abstract "topics" that occur in a collection of documents.
 Please refer to [wikipedia](https://en.wikipedia.org/wiki/Topic_model#:~:text=In%20machine%20learning%20and%20natural,structures%20in%20a%20text%20body.) for detailed information.
- These documents were crawled from *twitter* with some *COVID-19* related keywords . All tweets are in English but have a lot of noisy information.
- In this problem, your goal is to cluster tweets according to the latent 'topics' with the statistical method, automatically. 


### Load the csv file using pandas library

In [ ]:
#df = pd.read_csv('../data/covid_twitter.csv') #set your working directory
df = pd.read_csv("covid_twitter.csv", sep=",")

text_en = df['text']
df.head()

### Problem 4-1 Text Preprocessing
First of all, you should process the documents so that the model will not be perturbed by the noisy information

**a. Remove the following characters with *'re'* module in each tweets**
- special characters ( # @ , . | ? ! )
- *COVID* relavant words (COVID19, coronavirus, coronaoutbreak)

In [ ]:
### YOUR CODE HERE ###
#Hint. repeat first sentence process but change parameter to fit your gal
text_en = text_en.apply(lambda x: re.sub(r"[\#\@\,\.\d\'\|\(\)\-]", "", str(x)))
text_en = text_en.apply(lambda x: re.sub("COVID|COVID19|coronavirus|coronaoutbreak|covid|corona|Corona|Covid", "", str(x)))
text_en[:10]

**b. Remove the *stop words* in each tweets**

*Stop words* are words which should be filtered out before given to the model. Please refer to [wikipedia](https://en.wikipedia.org/wiki/Stop_words#:~:text=In%20SEO%20terminology%2C%20stop%20words,save%20space%20in%20their%20databases.) for detailed information 
- First, you should download the stop word set from *nltk* module.

In [ ]:
### NLP toolkit ###
import nltk
from nltk.corpus import stopwords

In [ ]:
### Download the set with this
nltk.download('stopwords')

In [ ]:
### check the set with this
stop_words = set(stopwords.words('english'))
print(stop_words)

**a. Update your set with some stop words which are not needed for modeling, you think.**

In [ ]:
### YOUR CODE HERE ###
#Hint in the paremeter '' input your stop_words
stop_words.update([''])
stop_words

**b. Remove stop words from the tweet dataset with your own stop words set**

In [ ]:
### YOUR CODE HERE ###
from nltk.tokenize import word_tokenize 
nltk.download("punkt", quiet=True)
word_tokens = []
for text in text_en:
    result = ""
    row = word_tokenize(text )
    
    for w in row: 
        if w not in stop_words: 
            result = result +" " +str(w) 
            
    word_tokens.append(result)

word_tokens[:5]

### Problem 4-2 Convert the documents to a tf-idf matrix 
*TF-IDF* is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus. Please refer to [this](http://mlwiki.org/index.php/TF-IDF)
- Please use the results of 4-1
- First, transform your tweets dataset into tf-idf matrix using the module below
- Print your tf-idf matrix after converting it into *pandas.DataFrame* format
- Set your own hyperparams.

In [ ]:
### scikit learn ###
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
### YOUR CODE HERE ###
tfidfv = TfidfVectorizer(min_df = 5,
    max_df = 0.95,
    max_features = 8000,
    stop_words = 'english').fit(word_tokens)
words = tfidfv.transform(word_tokens)
#print(tfidfv.vocabulary_)

### Problem 4-3 Cluster the tf-idf matrix of tweets using the *K-Means* algorithm.
- Set your own hyperparams

In [ ]:
### scikit learn ###
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

**a. Find an optimal value for the number of clusters using the *elbow method***
- Draw the plot for SSD(Sum of the Squared Distance) of *K-Means* algorithm by the number of clusters, K, as shown in the example


Example
![image.png](attachment:image.png)

In [ ]:
### YOUR CODE HERE ###
sse = []
for k in range(5, 105, 5):
    sse.append(MiniBatchKMeans(n_clusters=k, init_size=1024, batch_size=2048, random_state=0).fit(words).inertia_)
        
f, ax = plt.subplots(figsize=(15, 6))
ax.plot(x, sse, marker='^')
ax.set_xlabel('Number of Clusters')
ax.set_xticks(x)
ax.set_xticklabels(x)
ax.set_ylabel('Sum of Square Distance')
ax.set_title('Elbow Method')

**b. Based on the your plot above, pick a proper value for K and justify why.**

It is OK that you can't find any proper value for K. If not, specify why these are not proper.

**### YOUR DESCRIPTION HERE ###**
 - K=40 일때 Distnace 가 꾸준히 줄어들다가  그 이후 횡보하기 때문에 40일때가 적절하다고 하다고 판단된다.
 

### Problem 4-4 Visualize your clusters to evaluate the results of clustering analysis.

**a. Assgin the cluster index for each tweet with K=14**
- In other words, predict the label for each example

In [ ]:
# 14개의 그룹으로 나누는 K-Means 모델을 생성합니다
model = MiniBatchKMeans(n_clusters=14, init_size=1024, batch_size=2048, random_state=20)
model.fit(words)
labels = model.predict(words)

**b. Fill the function in below, which visualizes your clusters with PCA and TSNE**

Both *PCA* and *TSNE* are the most common visualization techniques in the machine learning community. It is usually known that TSNE could capture the non-linear relationship between datapoints than PCA. For more detailed information, please refer to [this](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding)
- At first, this function randomly selects some samples for fitting PCA, depending on your device's capability.
- Get the PCA matrices from the results of 4-2(tf-idf matrix) where the number of principal components is 2, each.
- If you are uncomfortable with this function, you can make your own function to accomplish the same goal with this.

Example
![image.png](attachment:image.png)

In [ ]:
### visualization methods ###
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
### for plotting ###
import matplotlib.cm as cm

In [ ]:
def plot_tsne_pca(data, labels):
    '''
    This function plots the PCA and t-SNE on 2D plane.
    args:
        data: tf-idf weight matrix
        labels: predictions from K-means
    '''
    # initial set up and random pick up samples
    max_label = max(labels)
    max_items = np.random.choice(range(data.shape[0]), size=2000, replace=False)
    
    ######################## YOUR CODE HERE ########################
    '''
    pca (2000 x 2) : extract 2 eigenvectors that have the most explained variance
    tsne (2000 x 2) : feed the 50 principal components to t-SNE
                     For TSNE, you can use the TSNE().fit_transform() function.
    '''
    
    pca = PCA(n_components=2).fit_transform(data[max_items, :].todense())
    tsne = TSNE().fit_transform(PCA(n_components=50).fit_transform(data[max_items,:].todense()))
    
    #################################################################
    
    # random pick certain size of data points for visiualization
    idx = np.random.choice(range(pca.shape[0]), size=400, replace=False)
    label_subset = labels[max_items]
    label_subset = [cm.hsv(i/max_label) for i in label_subset[idx]]
    
    f, ax = plt.subplots(1, 2, figsize=(14, 6))
    
    # plot PCA
    ax[0].scatter(pca[idx, 0], pca[idx, 1], c=label_subset)
    ax[0].set_title('PCA Cluster')
    
    # plot t-SNE
    ax[1].scatter(tsne[idx, 0], tsne[idx, 1], c=label_subset)
    ax[1].set_title('TSNE Cluster')

In [ ]:
### YOUR CODE HERE ###
# plot PCA and t-SNE reduced data with above function
plot_tsne_pca( words, labels)

### Problem 4-5 Extract keywords from each clusters

**a. get the top keywords from each cluster you've got from 4-4 (a.), based on the tf-idf matrix values**

- Use the function in below or make your own function to achieve the same goal with this function.

Example
![image.png](attachment:image.png)

In [ ]:
def get_top_keywords(data, clusters, labels, n_terms=10):
    '''
    This function displays the top keywords based on tf-idf score.
    data = tf-idf array
    clusters = assigned cluster index
    labels = vectorizer.get_feature_names()
    n_terms = top n keywords
    '''
    #  based on predictions
    df = pd.DataFrame(data.todense()).groupby(clusters).mean()
    
    # loop through each clusters and print top 10 score words
    for i,r in df.iterrows():
        print('\nCluster {}'.format(i))
        print(','.join([labels[t] for t in np.argsort(r)[-n_terms:]]))

In [ ]:
### YOUR CODE HERE ###
get_top_keywords(words, labels, tfidfv.get_feature_names(), 10)


**b. According to the keywords selected from (a.), describe the differences between clusters or guess the topic of tweets in the specific cluster**

**### YOUR DESCRIPTION HERE ###**

Cluster 1
german,finance,state,dont,suicide,people,need,minister,said,know

Cluster 2
mental,support,people,public,thank,healthcare,care,workers,health,pandemic

Cluster 3
china,need,virusoutbreak,like,stayathomeandstaysafe,world,time,new,update,people

- Cluster 1는 금융, 주, 자살, 도움, 장관의 단어들이 크러스터링 되고, 
  Cluster 2는 멘탈, 지원, 사람, 공적, 감사, 헬스케어, 케어 등 
  
  

### End of HW4 ###
Thanks for all your works